In [13]:
import time
import os
import json
import csv
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm_notebook

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [15]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

In [16]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict


In [17]:
def input_keyword(keyword):
    """ (string) -> string
    Args:
        keyword(string): 검색할 '단어'
    Return
        url(string)
        keyword에 따라 변화하는 url
    """
    url = "https://www.indeed.com/jobs?q=&l=" + keyword + "&start="
    return url
    

In [18]:
def move_page(url, keyword):
    """ (string, tring) -> None
    Args:
        url(string): input_keyword의 Return 값 (= keyword를 추가한 기본 url)
        keyword(string): 검색할 '단어'
    Return:
        None
        기능 설명: 페이지를 이동하면서 정보 수집(get_info())
    """
    page = input("총 몇 페이지까지 크롤링할까요?, 10단위로 입력바랍니다. ")
    for p in range(0, int(page), 10):
        print(url+str(p), '크롤링 시작')
        get_info(url+str(p), keyword)
        
 

In [19]:
def get_info(url, keyword):
    """ (string, string) -> None
    Args:
        url(string): input_keyword의 Return 값에 'page number' 더한 값 (keyword 및 page number 추가)
        keyword(string): 검색할 '단어'
    Return:
        None
        수집한 정보를 list 저장 -> csv 파일 작성 단계로 수행(write_csv 이용)
    
    """
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    corp_list = soup.select("div.jobsearch-SerpJobCard")
    
    for idx, corp in tqdm_notebook(enumerate(corp_list)):
        corp_cnt = list()
        try:
            job_title = corp.select("h2.title > a")[0].get_text().replace("\n", "")
            detail_url = 'https://www.indeed.com' + corp.select("h2.title > a")[0]['href']
            detail_req = requests.get(detail_url)
            detail_cont = detail_req.content
            detail_soup = BeautifulSoup(detail_cont, 'html.parser')
            detail_info = detail_soup.select_one("#jobDescriptionText").get_text()
            
        except Exception as e:
            print(detail_url, "에러 원인: ", e)
            detail_info = "CHECK"
        try:
            company_name = corp.select("span.company")[0].get_text().replace("\n", "")
        except:
            company_name = "NULL"
        try:
            location = corp.select("span.location")[0].get_text().replace("\n", "")
        except:
            loaction = "NULL"
        
        try:
            minus_date = corp.select("span.date ")[0].get_text().replace("\n","").split(" ")[0]
            try:
                minus_date = int(minus_date)
                minus_date = str(date.today() + relativedelta(days=-minus_date))
            except:
                if minus_date == 'Today' or 'Just':
                    minus_date = str(date.today())
                else:
                    minus_date = 'at least a month ago'
        except:
            minus_date = "NULL"
            
        try:
            short_descript = corp.select("div.summary")[0].get_text().replace("\n","").replace("\n","")
        except:
            short_descript = "NULL"
        
        corp_cnt.append(job_title)
        corp_cnt.append(company_name)
        corp_cnt.append(location)
        corp_cnt.append(minus_date)
        corp_cnt.append(short_descript)
        corp_cnt.append(detail_info)
        corp_cnt.append(detail_url)
        write_csv(str(keyword) + "_indeed.csv", corp_cnt)


In [20]:
def write_csv(filename, info_list):
    """ (string, list) -> None
    Args:
        filename(string): 파일 이름
        info_list(list): 저장할 정보
    Retrun:
        None
        csv파일에 info_list 작성
    """
    
    csv_file = open(filename, "a", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(csv_file)
    wr.writerow(info_list)
    csv_file.close()

In [21]:
def main():
    keyword = input("검색 키워드를 입력하세요")
    
    filename = keyword + '_indeed'
    
    # 파일이름 중복된 것이 있는지 확인하는 매개변수
    yesorno = False;
    # 디렉토리 내의 파일을 반복문으로 점검
    for tempFileName in os.listdir():
        # 디랙토리 내의 파일 중 csv로 끝나는 것만 확인
        if tempFileName.endswith('.csv'):
            # 중복되는 것이 있으면 True
            if filename == tempFileName:
                yesorno = True
    
    # 파일 이름이 같은 것이 있는 경우 -> pass
    if yesorno == True:
        pass
    # 파일 이름이 같은 것이 없는 경우 -> 파일 생성 및 헤더파일 작성
    else:
        csv_file = open(str(filename), "w", encoding = 'utf-8-sig', newline = '')
        wr = csv.writer(csv_file)
        wr.writerow(['job_title', 'company_name', 'location', 'minus_date', 'short_descript', 'detail_info', 'detail_url'])
        
    
    url = input_keyword(keyword)
    move_page(url, keyword)

## 1. 검색 키워드 입력
## 2. 찾고자하는 페이지 수 입력 (10 단위, 일반 데스크탑으로 작업할 시 최대 500 권장)
## 3. 세부정보 오류났을 시 -> 'CHECK' 저장 -> "CHECK" 검색 후 "detail_url'에서 재수집 권장

In [23]:
%%time
if __name__ == "__main__":
    main()

검색 키워드를 입력하세요ny
총 몇 페이지까지 크롤링할까요?, 10단위로 입력바랍니다. 700
https://www.indeed.com/jobs?q=&l=ny&start=0 크롤링 시작



https://www.indeed.com/jobs?q=&l=ny&start=10 크롤링 시작



https://www.indeed.com/jobs?q=&l=ny&start=20 크롤링 시작



https://www.indeed.com/jobs?q=&l=ny&start=30 크롤링 시작



https://www.indeed.com/jobs?q=&l=ny&start=40 크롤링 시작


...


https://www.indeed.com/jobs?q=&l=ny&start=660 크롤링 시작



https://www.indeed.com/jobs?q=&l=ny&start=670 크롤링 시작



https://www.indeed.com/jobs?q=&l=ny&start=680 크롤링 시작



https://www.indeed.com/jobs?q=&l=ny&start=690 크롤링 시작



Wall time: 34min 34s
